In [2]:
# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from dash import no_update
# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "nick"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

#FIX ME Add in Grazioso Salvare’s logo (DONE)
image_filename = 'gsLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
#(DONE)
app.layout = html.Div([
    # html.Div(id='debug-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
             style={'width': '25%', 'height': 'auto', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}),
    html.Div('Dashboard by Nick Allen', style={'text-align':'center', 'margin':'10px'}),
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc. (DONE)
    html.Div([
        html.Label('Interactive Filter Options'),
        dcc.Checklist(
            id='filter-checklist',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DIT'}
            ],
            value=[],  # the default is none selected
            labelStyle={'display': 'inline-block'}
        ),
        html.Button('Reset', id='reset-button', n_clicks=0)
    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable='single',),
    #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    #If you completed the Module Six Assignment, you can copy in the code you created here 
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display' : 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-checklist', 'value'),
     Input('reset-button', 'n_clicks')],
    prevent_initial_call=True  # Prevents the callback from running before the app starts
)

    ## FIX ME Add code to filter interactive data table with MongoDB queries
    #
    #        
    #        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    #        data=df.to_dict('records')
    #       
    #       
    #        return (data, columns) 
# (DONE)

def update_datatable(filter_values, n_clicks):
    # If the reset button has been clicked, return the unfiltered data
    if n_clicks > 0:
        return df.to_dict('records')

    # If no filters are selected, return the unfiltered data
    if not filter_values:
        return df.to_dict('records')

    # Start with the full dataset
    filtered_df = df

    # Apply filters - if any filter is selected, we construct the query accordingly
    if 'WR' in filter_values:
        water_rescue_query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        filtered_df = filtered_df[filtered_df.apply(lambda x: all([
            x['breed'] in water_rescue_query['breed']['$in'],
            x['sex_upon_outcome'] == water_rescue_query['sex_upon_outcome'],
            26 <= x['age_upon_outcome_in_weeks'] <= 156
        ]), axis=1)]

    if 'MWR' in filter_values:
        mountain_rescue_query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                              "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        filtered_df = filtered_df[filtered_df.apply(lambda x: all([
            x['breed'] in mountain_rescue_query['breed']['$in'],
            x['sex_upon_outcome'] == mountain_rescue_query['sex_upon_outcome'],
            26 <= x['age_upon_outcome_in_weeks'] <= 156
        ]), axis=1)]

    if 'DIT' in filter_values:
        disaster_tracking_query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                              "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
        filtered_df = filtered_df[filtered_df.apply(lambda x: all([
            x['breed'] in disaster_tracking_query['breed']['$in'],
            x['sex_upon_outcome'] == disaster_tracking_query['sex_upon_outcome'],
            20 <= x['age_upon_outcome_in_weeks'] <= 300
        ]), axis=1)]

    # Return the filtered records
    return filtered_df.to_dict('records')
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('filter-checklist', "value")])
def update_graphs(viewData, filter_values):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #] (DONE)
    # If filter_values is empty, it means no filters are selected
    if not filter_values:
        # Return a message indicating no data is available
        return html.Div("Select a filter to view the pie chart.")
    # If viewData is None or empty, we don't create a pie chart
    if not viewData:
        return no_update

    # Convert the viewData (which is in dictionary format) to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # If the DataFrame is empty, return an empty graph
    if dff.empty:
        return dcc.Graph(figure=px.pie())

    # Create the pie chart figure using Plotly Express
    fig = px.pie(dff, names='breed', title='Distribution of Animal Breeds')

    # Return the Graph component with the figure
    return dcc.Graph(figure=fig)
# This callback will highlight a cell on the data table when the user selects it
    @app.callback(
        Output('datatable-id', 'style_data_conditional'),
        [Input('datatable-id', 'selected_columns')]
    )
    def update_styles(selected_columns):
        return [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # Check if viewData is None or empty, which means there is no data to display
    if not viewData:
        return html.Div("No data available.")  # Display a message in the app

    # Check if the index is empty, which means no row is selected
    if not index:
        return html.Div("No row selected.")  # Display a message in the app

    # Convert the viewData to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because only single row selection is allowed, we take the first item in index
    row = index[0]

    # Extract latitude and longitude from the DataFrame
    latitude = dff.iloc[row, 13]  # 13 is the latitude column index
    longitude = dff.iloc[row, 14]  # 14 is the longitude column index

    # Extract additional information for the tooltip and popup
    breed = dff.iloc[row, 4]  # 4 is the breed column index
    name = dff.iloc[row, 9]  # 9 is the name column index

    # Create the map with the marker at the selected row's coordinates
    return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
        dl.TileLayer(url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png"),  # Added url parameter for tile layer
        dl.Marker(position=[latitude, longitude], children=[
            dl.Tooltip(f'Breed: {breed}'),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(name)
            ])
        ])
    ])



app.run_server(debug=True, mode='external')


Connection Successful
